In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import statsmodels.api as api
import time
import warnings
import os

warnings.filterwarnings('ignore')

In [4]:
#factors = 'ret_month idiovol beta bm illiq illiq_std log_cap mom str'.split()
old_path2 = 'D:/jupyterfile/empirical/数据/控制变量/'

lscolumns=[]
for i in os.listdir(old_path2):
    i=i[:-4]
    lscolumns.append(i)


factors = ['ret_month','RSJ']+lscolumns[1:]
data = pd.read_csv('D:/jupyterfile/empirical/数据/all_data/RSJ.csv',index_col=[0,1])[factors]
data = data.groupby(['date','code']).mean().unstack()
data

ret_month                                                    \
code        1.0       2.0       4.0       5.0       6.0       7.0        
date                                                                     
2001-01-31       NaN       NaN       NaN -0.095606 -0.047078       NaN   
2001-02-28       NaN  0.096909       NaN  0.111809  0.067663       NaN   
2001-03-31       NaN -0.038208       NaN -0.095893  0.021715       NaN   
2001-04-30       NaN       NaN       NaN       NaN -0.028374       NaN   
2001-05-31       NaN  0.041102       NaN -0.034914 -0.002573       NaN   
...              ...       ...       ...       ...       ...       ...   
2020-07-31       NaN       NaN -0.109330  0.005608 -0.229502  0.054573   
2020-08-31       NaN       NaN -0.098583 -0.123690       NaN  0.034001   
2020-09-30       NaN       NaN  0.150773 -0.020621 -0.111661 -0.058688   
2020-10-31       NaN       NaN  0.037105  0.018522  0.000507       NaN   
2020-11-30       NaN       NaN -0.251784 -0.020621 -0.022183       NaN   

                                                   ...       mom            \
code        8.0       9.0      10.0      11.0      ...  603987.0  603988.0   
date                                               ...                       
2001-01-31       NaN       NaN      NaN       NaN  ...       NaN       NaN   
2001-02-28       NaN       NaN      NaN       NaN  ...       NaN       NaN   
2001-03-31       NaN       NaN      NaN       NaN  ...       NaN       NaN   
2001-04-30       NaN       NaN      NaN       NaN  ...       NaN       NaN   
2001-05-31       NaN  0.010473      NaN       NaN  ...       NaN       NaN   
...              ...       ...      ...       ...  ...       ...       ...   
2020-07-31  0.002048 -0.084975      NaN       NaN  ...  1.263847  0.259886   
2020-08-31 -0.056979 -0.086208      NaN       NaN  ...  1.657692  1.085333   
2020-09-30 -0.067213  0.078759      NaN -0.217926  ...  1.273082  0.789563   
2020-10-31  0.009911 -0.017114      NaN -0.013393  ...  1.057306  0.598401   
2020-11-30 -0.067417  0.010856      NaN -0.120633  ...  1.091620  0.360393   

                                                                       \
code        603989.0  603990.0  603991.0  603993.0 603996.0  603997.0   
date                                                                    
2001-01-31       NaN       NaN       NaN       NaN      NaN       NaN   
2001-02-28       NaN       NaN       NaN       NaN      NaN       NaN   
2001-03-31       NaN       NaN       NaN       NaN      NaN       NaN   
2001-04-30       NaN       NaN       NaN       NaN      NaN       NaN   
2001-05-31       NaN       NaN       NaN       NaN      NaN       NaN   
...              ...       ...       ...       ...      ...       ...   
2020-07-31  0.535031  0.729393  0.540142  0.025692      NaN -0.082390   
2020-08-31  0.761378  1.257147  1.216259  0.239988      NaN -0.102688   
2020-09-30  0.593888  1.078208  1.236731  0.189601      NaN -0.091052   
2020-10-31  0.499967  0.536849  1.143608  0.106937      NaN -0.048855   
2020-11-30  0.351712  0.848010       NaN  0.122550      NaN  0.024518   

                                
code        603998.0  603999.0  
date                            
2001-01-31       NaN       NaN  
2001-02-28       NaN       NaN  
2001-03-31       NaN       NaN  
2001-04-30       NaN       NaN  
2001-05-31       NaN       NaN  
...              ...       ...  
2020-07-31  0.001477  0.122639  
2020-08-31 -0.013919  0.185846  
2020-09-30 -0.030275  0.211573  
2020-10-31       NaN -0.008526  
2020-11-30       NaN -0.114921  

[239 rows x 36870 columns]

In [5]:
def get_month_ret_and_cap(data,months=1):
    month_ret = data.ret_month.rolling(months).mean().shift(1-months)
    cap = np.exp(data.log_cap)
    return cap,month_ret

In [6]:
def get_group(df,q):
    group_list = []
    for i in range(len(q)-1):
        q1=q[i]
        q2=q[i+1]
        if q2==1:
            temp_group = df.apply(lambda x:(x>x.quantile(q1))&(x<=x.quantile(q2)),axis=1) * 1.0
        else:
            temp_group = df.apply(lambda x:(x>=x.quantile(q1))&(x<=x.quantile(q2)),axis=1) * 1.0
        temp_group = temp_group.replace(0,np.nan)
        group_list.append(temp_group)
    return group_list

In [7]:
def get_score(ret):
    pool = ret.copy()
    pool[~pool.isna()] = 1
    score = ((pool*cap).T/(pool*cap).sum(axis=1)).T
    return (score*ret).sum(axis=1)

def get_return_ser(f1,f2,type_):
    q = [0,0.3,0.7,1]
    groups = [group*f2 for group in get_group(f1,q)]
    q = [0,0.2,0.4,0.6,0.8,1]
    groups = [get_group(group,q) for group in groups]
    result = []
    
    if type_=='eq':
        for i in range(3):
            x = []
            for j in range(5):
                x.append((groups[i][j] * month_ret).mean(axis=1))
            x = pd.concat(x,axis=1)
            x[5] = x.iloc[:,-1]-x.iloc[:,0]
            result.append(x)
    
    if type_=='os':
        for i in range(3):
            x = []
            for j in range(5):
                x.append(get_score(groups[i][j] * month_ret))
            x = pd.concat(x,axis=1)
            x[5] = x.iloc[:,-1]-x.iloc[:,0]
            result.append(x)
    return result

In [8]:
def cal_double_sort_result(group_ret):
    result = []
    t = []
    p = []
    for group in group_ret:
        result.append(group.mean().round(4)*100)
        t.append(st.ttest_1samp(group,0)[0])
        p.append(st.ttest_1samp(group,0)[1])

    result = pd.DataFrame(result).applymap(lambda x:'{:.2f}'.format(x))
    t = pd.DataFrame(t).applymap(lambda x:'({:.2f})'.format(x))
    p = pd.DataFrame(p)
    p_ = p.astype('object').copy()
    p_[p>0.1] = ''
    p_[p<=0.1] = '*'
    p_[p<=0.05] = '**'
    p_[p<=0.01] = '***'
    p = p_.copy()
    
    for i in range(len(t)):
        result.iloc[i] = result.iloc[i].values + p.iloc[i].values
        
    result = result.append(t).sort_index()
    result.index = ['小','','中','','大','']
    result.columns = ['L','2','3','4','H','H-L']
    return result

In [9]:
cap,month_ret = get_month_ret_and_cap(data)
def get_result(f1,f2,type_):
    '''
    f1:控制变量
    f2:研究因子
    '''
    group_ret = get_return_ser(f1,f2,type_)
    result = cal_double_sort_result(group_ret)
    return result

In [10]:
result_lfull_eq = get_result(data.log_cap,data.RSJ,'eq')
result_lfull_os = get_result(data.log_cap,data.RSJ,'os')

# result_lfull = result_lfull_eq.append(result_lfull_os)

In [11]:
a = result_lfull_eq.append(result_lfull_os)

In [12]:
a

,L,2,3,4,H,H-L
小,1.93***,1.65**,1.54**,1.28*,0.72,-1.22***
,(2.86),(2.42),(2.22),(1.87),(1.05),(-6.70)
中,1.09*,0.99,0.86,0.66,0.38,-0.72***
,(1.72),(1.53),(1.35),(1.04),(0.61),(-3.31)
大,0.74,0.77,0.83,0.66,0.62,-0.11
,(1.28),(1.31),(1.40),(1.17),(1.08),(-0.39)
小,1.87***,1.57**,1.46**,1.23*,0.66,-1.20***
,(2.77),(2.31),(2.11),(1.80),(0.97),(-6.68)
中,1.05*,0.95,0.83,0.63,0.41,-0.64***
,(1.66),(1.49),(1.30),(1.00),(0.67),(-2.89)
